NOTE: Use Zipped "Data" folder, which has Training, Testing, Development folders. Rename Zip as Data.zip

In [838]:
import random
import numpy as np
import pandas as pd
import pprint, time

In [839]:
training_file = open("training.txt", "r", encoding= "utf8")
all_lines = training_file.read().split("\n")
training_file.close()
print(all_lines[:200])

['“\tPUNC', 'While\tCCS', 'much\tJJ', 'of\tADP', 'the\tDET', 'digital\tJJ', 'transition\tNN', 'is\tVAUX', 'unprecedented\tJJ', 'in\tADP', 'the\tDET', 'United\tNNP', 'States\tNNP', ',\tPUNC', 'the\tDET', 'peaceful\tJJ', 'transition\tNN', 'of\tADP', 'power\tNN', 'is\tVAUX', 'not\tRB', ',\tPUNC', '”\tPUNC', 'Obama\tNNP', 'special\tJJ', 'assistant\tNN', 'Kori\tNNP', 'Schulman\tNNP', 'wrote\tVM', 'in\tADP', 'a\tDET', 'blog\tNN', 'post\tNN', 'Monday\tNNP', '.\tPUNC', '', 'For\tADP', 'those\tPR', 'who\tPR', 'follow\tVM', 'social\tJJ', 'media\tNN', 'transitions\tNN', 'on\tADP', 'Capitol\tNNP', 'Hill\tNNP', ',\tPUNC', 'this\tPR', 'will\tVAUX', 'be\tVAUX', 'a\tDET', 'little\tJJ', 'different\tJJ', '.\tPUNC', '', 'But\tCCD', 'in\tADP', 'a\tDET', 'break\tNN', 'from\tADP', 'his\tPR', 'past\tJJ', 'rhetoric\tNN', 'about\tADP', 'curtailing\tVM', 'immigration\tNN', ',\tPUNC', 'the\tDET', 'GOP\tNNP', 'nominee\tNN', 'proclaimed\tVM', 'that\tCCS', 'as\tADP', 'president\tNN', 'he\tPR', 'would\tVAUX', 'allow

In [840]:
testing_file = open("english_ann.txt", "r", encoding= "utf8")
all_lines_test = testing_file.read().split("\n")
testing_file.close()
print(all_lines_test[:200])

['climate    NN', 'change    NN', 'worsens    VM', 'most    RB', 'infectious    PR', 'diseases    NN', 'of    ADP', 'the    DET', 'pathogens    NN', 'known    VM', 'to    ADP', 'have    VM', 'infected    VM', 'humans     NN', ',    PUNC', 'more    RB', 'than     ADP', 'half    RB', 'may    PR', 'cause    VM', 'widespread    PR', 'disease   NN', 'as    ADP', 'a    DET', 'result    NN', 'rising    VM', 'temperatures    NN', ',    PUNC', 'precipitation    NN', 'changes    NN', ',    PUNC', 'or    CCD', 'other    PR', 'climate    NN', '-    PUNC', 'related    PR', 'factors    NN', ',    PUNC', 'a    DET', 'study    NN', 'finds    VM', '.    PUNC', '', 'the    DET', 'consequences    NN', 'of    ADP', 'climate    NN', 'change    NN', '-    PUNC', 'including    ADP', 'warming    VM', ',    PUNC', 'drought    NN', ',    PUNC', 'and    CCONJ', 'sea    NN', '-    PUNC', 'level    NN', 'rise    NN', '-    PUNC', 'worsen    VM', 'most    RB', 'infectious    PR', 'diseases    NN', 'by    ADP', 'fac

Read data

In [841]:
data_set = []
cur_sentence = []
cur_word_tag = ()

for line in all_lines:
   #print(line)
    if(line == ""):
        data_set.append(cur_sentence)
        cur_sentence = []
    else:
        cur_word_tag = line.split("\t")
        #print(cur_word_tag)
        cur_sentence.append(cur_word_tag)
print(data_set[:10])

[[['“', 'PUNC'], ['While', 'CCS'], ['much', 'JJ'], ['of', 'ADP'], ['the', 'DET'], ['digital', 'JJ'], ['transition', 'NN'], ['is', 'VAUX'], ['unprecedented', 'JJ'], ['in', 'ADP'], ['the', 'DET'], ['United', 'NNP'], ['States', 'NNP'], [',', 'PUNC'], ['the', 'DET'], ['peaceful', 'JJ'], ['transition', 'NN'], ['of', 'ADP'], ['power', 'NN'], ['is', 'VAUX'], ['not', 'RB'], [',', 'PUNC'], ['”', 'PUNC'], ['Obama', 'NNP'], ['special', 'JJ'], ['assistant', 'NN'], ['Kori', 'NNP'], ['Schulman', 'NNP'], ['wrote', 'VM'], ['in', 'ADP'], ['a', 'DET'], ['blog', 'NN'], ['post', 'NN'], ['Monday', 'NNP'], ['.', 'PUNC']], [['For', 'ADP'], ['those', 'PR'], ['who', 'PR'], ['follow', 'VM'], ['social', 'JJ'], ['media', 'NN'], ['transitions', 'NN'], ['on', 'ADP'], ['Capitol', 'NNP'], ['Hill', 'NNP'], [',', 'PUNC'], ['this', 'PR'], ['will', 'VAUX'], ['be', 'VAUX'], ['a', 'DET'], ['little', 'JJ'], ['different', 'JJ'], ['.', 'PUNC']], [['But', 'CCD'], ['in', 'ADP'], ['a', 'DET'], ['break', 'NN'], ['from', 'ADP'], [

In [842]:
import re

#print(f"Length of all_lines_test: {len(all_lines_test)}")

data_set_test = []
cur_sentence_test = []
cur_word_tag_test = ()

for i, line in enumerate(all_lines_test):
    line_repr = ''.join(c if c.isprintable() else '�' for c in line)
    #print(f"Processing line {i}: {line_repr}")

    if not line.strip():
        #print("Empty line detected, appending sentence to data_set_test")
        data_set_test.append(cur_sentence_test)
        cur_sentence_test = []
    else:
        cur_word_tag_test = line.split("\t")
        #print(f"Appending word_tag to cur_sentence_test: {cur_word_tag_test}")
        cur_sentence_test.append(cur_word_tag_test)

# Check if cur_sentence_test is empty before appending to data_set_test
if cur_sentence_test:
    #print("Appending the last sentence to data_set_test")
    data_set_test.append(cur_sentence_test)

print("Final data_set_test:")
for sentence in range(len(data_set_test)):
  for word in range(len(data_set_test[sentence])):
    data_set_test[sentence][word]= re.split(r" +", data_set_test[sentence][word][0])
print(data_set_test[:10])

Final data_set_test:
[[['climate', 'NN'], ['change', 'NN'], ['worsens', 'VM'], ['most', 'RB'], ['infectious', 'PR'], ['diseases', 'NN'], ['of', 'ADP'], ['the', 'DET'], ['pathogens', 'NN'], ['known', 'VM'], ['to', 'ADP'], ['have', 'VM'], ['infected', 'VM'], ['humans', 'NN'], [',', 'PUNC'], ['more', 'RB'], ['than', 'ADP'], ['half', 'RB'], ['may', 'PR'], ['cause', 'VM'], ['widespread', 'PR'], ['disease', 'NN'], ['as', 'ADP'], ['a', 'DET'], ['result', 'NN'], ['rising', 'VM'], ['temperatures', 'NN'], [',', 'PUNC'], ['precipitation', 'NN'], ['changes', 'NN'], [',', 'PUNC'], ['or', 'CCD'], ['other', 'PR'], ['climate', 'NN'], ['-', 'PUNC'], ['related', 'PR'], ['factors', 'NN'], [',', 'PUNC'], ['a', 'DET'], ['study', 'NN'], ['finds', 'VM'], ['.', 'PUNC']], [['the', 'DET'], ['consequences', 'NN'], ['of', 'ADP'], ['climate', 'NN'], ['change', 'NN'], ['-', 'PUNC'], ['including', 'ADP'], ['warming', 'VM'], [',', 'PUNC'], ['drought', 'NN'], [',', 'PUNC'], ['and', 'CCONJ'], ['sea', 'NN'], ['-', 'PUNC

In [843]:
print("First two sentences from dataset:")
print(data_set[:2])

First two sentences from dataset:
[[['“', 'PUNC'], ['While', 'CCS'], ['much', 'JJ'], ['of', 'ADP'], ['the', 'DET'], ['digital', 'JJ'], ['transition', 'NN'], ['is', 'VAUX'], ['unprecedented', 'JJ'], ['in', 'ADP'], ['the', 'DET'], ['United', 'NNP'], ['States', 'NNP'], [',', 'PUNC'], ['the', 'DET'], ['peaceful', 'JJ'], ['transition', 'NN'], ['of', 'ADP'], ['power', 'NN'], ['is', 'VAUX'], ['not', 'RB'], [',', 'PUNC'], ['”', 'PUNC'], ['Obama', 'NNP'], ['special', 'JJ'], ['assistant', 'NN'], ['Kori', 'NNP'], ['Schulman', 'NNP'], ['wrote', 'VM'], ['in', 'ADP'], ['a', 'DET'], ['blog', 'NN'], ['post', 'NN'], ['Monday', 'NNP'], ['.', 'PUNC']], [['For', 'ADP'], ['those', 'PR'], ['who', 'PR'], ['follow', 'VM'], ['social', 'JJ'], ['media', 'NN'], ['transitions', 'NN'], ['on', 'ADP'], ['Capitol', 'NNP'], ['Hill', 'NNP'], [',', 'PUNC'], ['this', 'PR'], ['will', 'VAUX'], ['be', 'VAUX'], ['a', 'DET'], ['little', 'JJ'], ['different', 'JJ'], ['.', 'PUNC']]]


In [844]:
print("First two sentences from testing set:")
print(data_set_test[:2])

First two sentences from testing set:
[[['climate', 'NN'], ['change', 'NN'], ['worsens', 'VM'], ['most', 'RB'], ['infectious', 'PR'], ['diseases', 'NN'], ['of', 'ADP'], ['the', 'DET'], ['pathogens', 'NN'], ['known', 'VM'], ['to', 'ADP'], ['have', 'VM'], ['infected', 'VM'], ['humans', 'NN'], [',', 'PUNC'], ['more', 'RB'], ['than', 'ADP'], ['half', 'RB'], ['may', 'PR'], ['cause', 'VM'], ['widespread', 'PR'], ['disease', 'NN'], ['as', 'ADP'], ['a', 'DET'], ['result', 'NN'], ['rising', 'VM'], ['temperatures', 'NN'], [',', 'PUNC'], ['precipitation', 'NN'], ['changes', 'NN'], [',', 'PUNC'], ['or', 'CCD'], ['other', 'PR'], ['climate', 'NN'], ['-', 'PUNC'], ['related', 'PR'], ['factors', 'NN'], [',', 'PUNC'], ['a', 'DET'], ['study', 'NN'], ['finds', 'VM'], ['.', 'PUNC']], [['the', 'DET'], ['consequences', 'NN'], ['of', 'ADP'], ['climate', 'NN'], ['change', 'NN'], ['-', 'PUNC'], ['including', 'ADP'], ['warming', 'VM'], [',', 'PUNC'], ['drought', 'NN'], [',', 'PUNC'], ['and', 'CCONJ'], ['sea', '

In [845]:
# split data into training and validation set in the ratio 80:20
train_set,test_set = data_set, data_set_test

In [846]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ][:17000]
test_tagged_words = [ tup for sent in train_set for tup in sent ][17000:]

In [847]:
print("Length of Train set and test set:")

print(len(train_tagged_words))
print(len(test_tagged_words))

Length of Train set and test set:
17000
4316


In [848]:
# print some of the tagged words.
train_tagged_words[:5]

[['“', 'PUNC'],
 ['While', 'CCS'],
 ['much', 'JJ'],
 ['of', 'ADP'],
 ['the', 'DET']]

In [849]:
#use set datatype to check how many unique tags are present in training data
tags = set()
for word, tag in train_tagged_words:
    tags.add(tag)
print("Total tags:", len(tags))
print("All tags:", tags)
# print(tags)

Total tags: 18
All tags: {'PR', 'DET', 'ADP', 'VM', 'RB', 'NNP', 'NN', 'RP', 'QTC', 'X', 'SYM', 'VAUX', 'INTJ', 'PUNC', 'CCS', 'JJ', '_', 'CCD'}


In [850]:
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}


Define Emission Probability and Transition Probability

In [851]:
# compute Emission Probability
def emission(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)


    return (count_w_given_tag, count_tag)

# compute  Transition Probability
def transition(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [852]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)

# em_matrix = np.zeros((len(tags), len(vocab)))
# for i, tag in enumerate(tags):
#     for j, word in enumerate(vocab):
#         em_matrix[i, j] = emission(word, tag)[0]/emission(word, tag)[1]



tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = transition(t2, t1)[0]/transition(t2, t1)[1]


In [853]:
# print(em_matrix)
print(tags_matrix)

[[1.58150848e-02 9.73236002e-03 3.52798067e-02 2.78588802e-01
  4.62287106e-02 9.73236002e-03 1.71532840e-01 6.08272525e-03
  3.64963501e-03 0.00000000e+00 0.00000000e+00 2.88321167e-01
  0.00000000e+00 4.37956192e-02 1.21654500e-03 7.54257888e-02
  4.86618001e-03 9.73236002e-03]
 [2.51098559e-03 5.02197118e-03 5.02197118e-03 1.63214058e-02
  1.38104204e-02 1.28060266e-01 5.31701207e-01 0.00000000e+00
  1.94601379e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 6.27746386e-03 0.00000000e+00 2.56120533e-01
  1.44381672e-02 1.25549280e-03]
 [7.58835748e-02 3.69542629e-01 1.97505206e-02 8.83575901e-03
  1.66320167e-02 1.57484412e-01 1.73076928e-01 0.00000000e+00
  6.18503131e-02 5.19750523e-04 3.63825355e-03 5.19750523e-04
  0.00000000e+00 1.24740126e-02 2.07900209e-03 8.93970877e-02
  6.75675692e-03 1.55925157e-03]
 [6.90448806e-02 1.86996549e-01 2.57767558e-01 1.95627157e-02
  5.35097830e-02 3.73993106e-02 1.09321058e-01 5.46605289e-02
  1.49597237e-02 0.00000000e+00 2

In [854]:
# convert the matrix to a dataframe for better readability
# the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
print(tags_df)


            PR       DET       ADP        VM        RB       NNP        NN  \
PR    0.015815  0.009732  0.035280  0.278589  0.046229  0.009732  0.171533   
DET   0.002511  0.005022  0.005022  0.016321  0.013810  0.128060  0.531701   
ADP   0.075884  0.369543  0.019751  0.008836  0.016632  0.157484  0.173077   
VM    0.069045  0.186997  0.257768  0.019563  0.053510  0.037399  0.109321   
RB    0.056818  0.065341  0.095170  0.234375  0.098011  0.014205  0.024148   
NNP   0.010401  0.005944  0.071322  0.106984  0.018574  0.187221  0.084695   
NN    0.022093  0.008592  0.260816  0.081006  0.024241  0.019945  0.113225   
RP    0.002857  0.008571  0.005714  0.605714  0.034286  0.005714  0.160000   
QTC   0.000000  0.013550  0.102981  0.018970  0.013550  0.054201  0.292683   
X     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
SYM   0.000000  0.000000  0.225000  0.000000  0.000000  0.000000  0.025000   
VAUX  0.009877  0.085185  0.035802  0.419753  0.175309  0.008642

Define Viterbi Algorithm

In [855]:
def Viterbi(sentence, tag_probs=train_tagged_words):
    state_sequence = []
    tags = list(set([pair[1] for pair in tag_probs]))

    for index, word in enumerate(sentence):
        max_probability = float('-inf')
        max_probability_tag = None

        for tag in tags:
            if index == 0:
                transition_probability = 0
            else:
                transition_probability = tags_df.loc[state_sequence[-1], tag]

            emission_probability = emission(word, tag)[0] / emission(word, tag)[1]
            state_probability = emission_probability * transition_probability

            if state_probability > max_probability:
                max_probability = state_probability
                max_probability_tag = tag

        state_sequence.append(max_probability_tag)

    return list(zip(sentence, state_sequence))

Create Test set

In [856]:
test_run= data_set_test

# list of tagged words
test_run_base = [tuple(tup) for sent in test_run for tup in sent]
print("Test sentences:")
print(test_run_base)
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
# test_tagged_words = test_tagged_words[:100]

Test sentences:
[('climate', 'NN'), ('change', 'NN'), ('worsens', 'VM'), ('most', 'RB'), ('infectious', 'PR'), ('diseases', 'NN'), ('of', 'ADP'), ('the', 'DET'), ('pathogens', 'NN'), ('known', 'VM'), ('to', 'ADP'), ('have', 'VM'), ('infected', 'VM'), ('humans', 'NN'), (',', 'PUNC'), ('more', 'RB'), ('than', 'ADP'), ('half', 'RB'), ('may', 'PR'), ('cause', 'VM'), ('widespread', 'PR'), ('disease', 'NN'), ('as', 'ADP'), ('a', 'DET'), ('result', 'NN'), ('rising', 'VM'), ('temperatures', 'NN'), (',', 'PUNC'), ('precipitation', 'NN'), ('changes', 'NN'), (',', 'PUNC'), ('or', 'CCD'), ('other', 'PR'), ('climate', 'NN'), ('-', 'PUNC'), ('related', 'PR'), ('factors', 'NN'), (',', 'PUNC'), ('a', 'DET'), ('study', 'NN'), ('finds', 'VM'), ('.', 'PUNC'), ('the', 'DET'), ('consequences', 'NN'), ('of', 'ADP'), ('climate', 'NN'), ('change', 'NN'), ('-', 'PUNC'), ('including', 'ADP'), ('warming', 'VM'), (',', 'PUNC'), ('drought', 'NN'), (',', 'PUNC'), ('and', 'CCONJ'), ('sea', 'NN'), ('-', 'PUNC'), ('le

Predict the tags for test set using Viterbi algorithm

In [857]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Predicted sequence:")
print(tagged_seq)
print("Predicted sequence length:")
print(len(tagged_seq) )

print("Time taken in seconds: ", difference)

Predicted sequence:
[('climate', 'PR'), ('change', 'VM'), ('worsens', 'PR'), ('most', 'RB'), ('infectious', 'PR'), ('diseases', 'PR'), ('of', 'ADP'), ('the', 'DET'), ('pathogens', 'PR'), ('known', 'VM'), ('to', 'RP'), ('have', 'VM'), ('infected', 'PR'), ('humans', 'NN'), (',', 'PUNC'), ('more', 'RB'), ('than', 'ADP'), ('half', 'DET'), ('may', 'PR'), ('cause', 'VM'), ('widespread', 'PR'), ('disease', 'NN'), ('as', 'ADP'), ('a', 'DET'), ('result', 'NN'), ('rising', 'VM'), ('temperatures', 'NN'), (',', 'PUNC'), ('precipitation', 'NN'), ('changes', 'NN'), (',', 'PUNC'), ('or', 'CCD'), ('other', 'JJ'), ('climate', 'NN'), ('-', 'PUNC'), ('related', 'JJ'), ('factors', 'NN'), (',', 'PUNC'), ('a', 'DET'), ('study', 'NN'), ('finds', 'PR'), ('.', 'PUNC'), ('the', 'DET'), ('consequences', 'NN'), ('of', 'ADP'), ('climate', 'NN'), ('change', 'NN'), ('-', 'PUNC'), ('including', 'VM'), ('warming', 'NN'), (',', 'PUNC'), ('drought', 'NN'), (',', 'PUNC'), ('and', 'CCD'), ('sea', 'NN'), ('-', 'PUNC'), ('l

Calculate Accuracy

In [858]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

# accuracy
# check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]

# print(check)
# accuracy = len(check)/len(tagged_seq)
# print('Viterbi Algorithm Accuracy: ',accuracy*100)

first_tags = [v for k, v in test_run_base]
second_tags = [v for k, v in tagged_seq]

precision = precision_score(first_tags, second_tags, average="weighted", zero_division=0)

print("Precision:", precision)

recall = recall_score(first_tags, second_tags, average="weighted", zero_division=0)

print("Recall:", recall)

f1 = f1_score(first_tags, second_tags, average="weighted", zero_division=0)

print("F1 Score:", f1)

def confusion_matrix(actual, predicted):
    tags= list(set(actual))
    tags.sort()
    c= pd.DataFrame(np.zeros((len(tags), len(tags))), index=tags, columns=tags)
    for i in range(len(actual)):
        if actual[i] in c.index and predicted[i] in c.columns:
            c.loc[actual[i], predicted[i]]+= 1
    return c

c = confusion_matrix(first_tags, second_tags)

print(c)

Precision: 0.7464015114437349
Recall: 0.5869918699186992
F1 Score: 0.6290258781220798
        ADP   CCD  CCONJ   DET     NN  NUM  PART    PR  PRON  PROPN  PUNC  \
ADP    64.0   0.0    0.0   0.0    0.0  0.0   0.0   0.0   0.0    0.0   0.0   
CCD     0.0   1.0    0.0   0.0    0.0  0.0   0.0   0.0   0.0    0.0   0.0   
CCONJ   0.0  15.0    0.0   0.0    0.0  0.0   0.0   0.0   0.0    0.0   0.0   
DET     0.0   0.0    0.0  52.0    0.0  0.0   0.0   5.0   0.0    0.0   0.0   
NN      0.0   0.0    0.0   0.0  104.0  0.0   0.0  61.0   0.0    0.0   0.0   
NUM     0.0   0.0    0.0   0.0    1.0  0.0   0.0   4.0   0.0    0.0   0.0   
PART    5.0   0.0    0.0   0.0    0.0  0.0   0.0   0.0   0.0    0.0   0.0   
PR      0.0   0.0    0.0   0.0    2.0  0.0   0.0  20.0   0.0    0.0   0.0   
PRON    0.0   0.0    0.0   1.0    0.0  0.0   0.0   9.0   0.0    0.0   0.0   
PROPN   0.0   0.0    0.0   0.0    4.0  0.0   0.0  18.0   0.0    0.0   0.0   
PUNC    0.0   0.0    0.0   0.0    0.0  0.0   0.0   0.0   0.0    0.0